# INSTALL AND IMPORT NECESSARY MODULE, LIBRARY

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 24.5 MB/s eta 0:00:0

In [ ]:
import logging
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# READ DATA

In [ ]:
#đọc dữ liệu (nếu muốn làm task khác thì dẫn đường link tới dữ liệu của task đó.)

train_data = pd.read_csv("/content/drive/MyDrive/NHANDANG/DATA/FULL/full_TRAIN.csv")
eval_data = pd.read_csv("/content/drive/MyDrive/NHANDANG/DATA/FULL/full_DEV.csv")
test_data = pd.read_csv("/content/drive/MyDrive/NHANDANG/DATA/FULL/full_TEST.csv")


# PRECESSING FOR FINE-TUNE

In [ ]:
train_data['words'] = train_data['words'].astype(str)
train_data['labels'] = train_data['labels'].astype(str)
eval_data['words'] = eval_data['words'].astype(str)
eval_data['labels'] = eval_data['labels'].astype(str)

In [ ]:
train_data=train_data.dropna()

# MODEL

## Bert

In [ ]:
# Get unique labels from the label column in train_df
label_list = train_data["labels"].unique().tolist()

# Configure the model
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 40   # thay đổi số epoch tuỳ thích
model_args.learning_rate = 5e-5  #thay đổi số tuỳ thích
model_args.best_model_dir="/content/drive/MyDrive/NHANDANG/save_full_5e_5"  # nơi sẽ lưu folder model có loss vủa tập eval thấp nhất
model_args.overwrite_output_dir =True
model_args.train_batch_size = 16   # số lượng mẫu trong 1 lần học
model_args.eval_batch_size = 16   # số lượng mẫu trong 1 lần đánh giá
model_args.use_cached_eval_features = False
model_args.use_multiprocessing = False
model_args.reprocess_input_data = True
model_args.use_early_stopping = True
model_args.early_stopping_metric = "eval_loss"
model_args.early_stopping_metric_minimize = True
model_args.early_stopping_patience = 3
model_args.early_stopping_consider_epochs=True
model_args.evaluate_during_training=True
# Initialize the model
bert_model = NERModel(
    "bert",
    "bert-base-multilingual-cased",
    labels=label_list,
    args=model_args
)

# Train the model
# Train the model
bert_model.train_model(train_data, eval_data=eval_data)

result, model_outputs, preds_list = bert_model.eval_model(test_data)

In [ ]:
result

In [ ]:
# Make predictions with the model
sentences = ["đóng gói"]
predictions, raw_outputs = bert_model.predict(sentences)
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'đóng': 'O'}, {'gói': 'O'}]]